In [2]:
import pandas as pd
import numpy as np
import os
import re

import nltk
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/IW_student/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
os.getcwd()

'/home/IW_student/work/food_flavor/kochbar_ingredient_preprocessing/long_ingr_processing'

In [4]:
data=pd.read_csv('~/work/food_flavor/kochbar_ingredient_preprocessing/datasets/long_ingr_data/long_ingr_trans_003.csv',index_col=0)
data.shape

(7098, 5)

In [5]:
long_ingr=data[data['en_ingr'].str.split().str.len()>2]
long_ingr.shape

(1869, 5)

In [5]:
long_ingr.reset_index(drop=True,inplace=True)

In [6]:
long_ingr.head()

,recipe_id,recipe_href,name,ingredient_name,en_ingr
0,86439,/rezept/254348/Gelbflossen-Thunfisch-Tranchen-...,gelb flossen thunfischtranchen a ca. 150 gramm,gelb flossen thunfischtranchen a,yellow fins tuna
1,45230,/rezept/242380/Zimt-Apfel-Kuchen.html,rosinen nach geschmackeingeweicht in rum,rosinen nach geschmackeingeweicht in rum,raisins soaked rum
2,57005,/rezept/281248/Chilicreme.html,frischkäse geht auch mitfeta käse,frischkäse geht auch mitfeta käse,cream cheese goes well feta cheese
3,53172,/rezept/405833/Weihnachtliche-Muffins-a-la-Kra...,milch zum anrühren derbuttercreme,milch zum anrühren derbuttercreme,milk mixing buttercream
4,85805,/rezept/448409/Nachtisch-Cranberry-Dessert-a-l...,weisse schokolade mit nuss,weisse schokolade mit nuss,white chocolate nuts


In [7]:
long_ingr[long_ingr['recipe_href']=='/rezept/373769/Scharfe-Chili_Sauce.html']

,recipe_id,recipe_href,name,ingredient_name,en_ingr


In [7]:
long_ingrs=list(set(long_ingr['en_ingr'].values.tolist()))
len(long_ingrs)

1598

In [8]:
long_ingrs.sort()
long_ingrs

['0 chocolate couverture',
 '0 skin  bones',
 '5 minutes 180 degrees',
 '6 water soup',
 '7 gr peanut bars snickers',
 '8 ° gradwarm',
 'accompaniment crust roast',
 'aepfelwo bake well',
 'african peppers cheese mango',
 'african zulu dipp',
 'alba oil rapeseed oil butter flavor',
 'alba oil rewe',
 'alcohol cherioben 6 pcs',
 'almond flakes tight',
 'almond flour ger almonds rewe',
 'almonds cake border',
 'almost forgotten onions vegetables',
 'alpine milk chocolate yogurt garnish',
 'alpro drink calcium',
 'aluminum foil wrap asparagus',
 'amarena cherries juice',
 'american cookies ausdem spuermarkt',
 'anchovies fillet olive oil',
 'anise amp brown cane sugar',
 'anise use spices',
 'apple boskop elstar',
 'apple juice 100 cabbage',
 'apple juice refine sauce',
 'apples boskop depending',
 'apples preferably sweet variety pink lady',
 'approximate lemon zest',
 'apricot jelly topping',
 'arctic mustard honey finland',
 'aroma stock seasonings',
 'aromatic apples bread boskop',
 '

In [9]:
# replace x22,x2c, delete x

In [10]:
long_ingr['en_ingr']=long_ingr['en_ingr'].apply(lambda x:x.replace('x22',''))
long_ingr['en_ingr']=long_ingr['en_ingr'].apply(lambda x:x.replace('x2c',''))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
del_verbs=['add','request','scraped','grind','sliced','dice','diced','peeled','needs','depend','depends',
         'canned','cooked','marinated','beloved','blanched','cut','boil','boiled','boiling','minced',
        'broken','chopped','chop','grated','ground','contains','product','cored','weighed','drained',
         'crumbled','dried','see','filled','fine-grained','finished','pointed','frozen','pressed','desired',
         'gratinated','prepared','trust','happened','homemade','raised','pickled','trust','strainer',
         'refrigerated','dusted','loaded','made','make','mark','mashed','melted','mixed','needs','halved',
         'required','peeled','processed','untreated','bring','cleaned','steamed','crushed','making','whole',
          'open','rubbed','forgot','freeze-dried','frozen','fussy','liked','made','added','blended','eaten',
          'is','makes','stir','purchased','cored,','peel',' soaked','fried','frying','garnishing','grained',
           'likes','avoid','absolutely']
del_nouns=['addition','additional','direction','teaspoon','whole','pith','amount','pinch',
          'andreas','leftover','leftovers','strips','strip','cube','cubes','bit','mill','bottle','taste',
         'package','spread','couple','cubes','cultivation','slices','slice','half','rings','recipe',
         'choice','produced in-house','eighth','eye','hurry','shelf','rings','ring','part','rest','parts',
         'glass','recipe','kd','tablespoon','teaspoon','little','wrestle','toothpicks','at will',
        'confectioner','pieces','above','wish','size','garden','example','waxy','form','bold','shape',
          'option','girls','gravy','tipp','floor','the','drops','everything','everybody','bit','back',
          'absence']
del_ads=['already','alternatively','extra','fresh','freshly','fine','finely','small','coarse','possible',
        'always','even','evenly','desired','large','big','thick','tinn','according',
        'available','best','young','old','coarse','coarsely','colorful','dry','dark','enough','exclusively',
       'fat','giant','hand full','handful','lack','lean','less','straight','little','advance','cold',
       'low-fat','many','medium-sized','mini','easily','easy','much','mulled','necessary','nicely','normal',
       'optional','optionally','different','otherwise','curly','smooth','possibly','possible','ready-to-cook',
       'ready-to-bake','ready-made','roughly','briefly','sharp','thinly','thin','alternatively','alternative',
        'another','arbitrary','soft','best','but','not','here','minus','enough','good','seasonally',
        'exactly','very','ready','roughly','beforehand','mild','slightly','fruity','another','skinless','approx',
        'approximate','depending']

In [12]:
del_words=del_verbs+del_nouns+del_ads

In [13]:
def del_word(ingredient):
    no_word=' '.join([c for c in ingredient.split() if c not in del_words])
    return no_word

In [14]:
long_ingr['en_ingr']=long_ingr['en_ingr'].apply(lambda x:del_word(x))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
long_ingrs.sort()
long_ingrs

['0 chocolate couverture',
 '0 skin bones',
 '5 minutes 180 degrees',
 '6 water soup',
 '7 gr peanut bars snickers',
 '8 herbs x',
 '8 ° gradwarm',
 'accompaniment crust roast',
 'aepfelwo bake well',
 'african peppers cheese mango',
 'african zulu dipp',
 'alba oil rapeseed oil butter flavor',
 'alba oil rewe',
 'alcohol cherioben 6 pcs',
 'almond flakes tight',
 'almond flour ger almonds rewe',
 'almonds cake border',
 'almost forgotten onions vegetables',
 'alpine milk chocolate yogurt garnish',
 'alpro drink calcium',
 'aluminum foil wrap asparagus',
 'amarena cherries juice',
 'american cookies ausdem spuermarkt',
 'anchovies fillet olive oil',
 'anise amp brown cane sugar',
 'anise use spices',
 'apple boskop elstar',
 'apple juice 100 cabbage',
 'apple juice refine sauce',
 'apples boskop',
 'apples preferably sweet variety pink lady',
 'apricot jelly topping',
 'arctic mustard honey finland',
 'aroma stock seasonings',
 'aromatic apples bread boskop',
 'around kirsch water cake

In [20]:
long_ingr[long_ingr['en_ingr']=='almost forgotten onions vegetables']

,recipe_id,recipe_href,name,ingredient_name,en_ingr
1857,1873,/rezept/405838/Forelle-in-Butter-gebraten-Salz...,zwiebel fürs gemüse fastvergessen,zwiebel fürs gemüse fastvergessen,almost forgotten onions vegetables


In [18]:
a_ingr=[]
for index,row in long_ingr.iterrows():
    if re.search(r'^a',row.en_ingr)!=None:
        a_ingr.append(row.en_ingr)
a_ingr=list(set(a_ingr))

In [19]:
a_ingr

['apples boskop',
 'apple juice refine sauce',
 'almonds cake border',
 'anchovies fillet olive oil',
 'asian sweet soursauce',
 'african peppers cheese mango',
 'aroma stock seasonings',
 'african zulu dipp',
 'alpro drink calcium',
 'alpine milk chocolate yogurt garnish',
 'almost forgotten onions vegetables',
 'around kirsch water cake base',
 'almond flakes tight',
 'apples preferably sweet variety pink lady',
 'accompaniment crust roast',
 'anise amp brown cane sugar',
 'asia vegetables zbiglo',
 'apple juice 100 cabbage',
 'amarena cherries juice',
 'alba oil rewe',
 'asparagus white demglas unfortunately freshness house',
 'alba oil rapeseed oil butter flavor',
 'alcohol cherioben 6 pcs',
 'american cookies ausdem spuermarkt',
 'almond flour ger almonds rewe',
 'asia ketjap sweet sauce',
 'asia wok vegetables iglo',
 'apricot jelly topping',
 'anise use spices',
 'asparagus shells 500 g asparagus',
 'arctic mustard honey finland',
 'aepfelwo bake well',
 'aluminum foil wrap aspa

In [ ]:
# delete whole
# ' depending season','0 skin  bones', '5 minutes 180 degrees','8 ° gradwarm','aepfelwo bake well','cannot recommend brunch',
# 'carefully turn clamp','cd cool music mom','decoration decorate diepupse','details alsoca information',
# 'don39t need nuts','don39t use breadcrumbs meat farce','drop red paint','edelgard please forgive','felt pens toddlers',
# 'flan muss 60minuten garen','ideally 5 minutes','kitchen roll towel','least 200g pulp','accompaniment crust roast',
# 'around kirsch water cake base',

In [ ]:
# part delete
# 'aluminum foil wrap','aluminum foil wrap','goes well','right consistency','mix well mixer','muffin tin 12 cavities','needed shortly serving','cooking bag',
# 'ausdem spuermarkt'

In [ ]:
# delete word
# 'day','x','mediumsized','zbbeckers','seedlings','amp',

In [ ]:
# load one_ingr and two ingr

In [10]:
one_ingr=pd.read_csv('~/work/food_flavor/kochbar_ingredient_preprocessing/datasets/long_ingr_data/one_ingr_data/one_ingr_done.csv',index_col=0)
one_ingr.shape

(2449, 4)

In [11]:
one_ingr.head()

,recipe_id,recipe_href,name,end_ingr
0,80,/rezept/438956/Nudelauflauf-mit-Gemuese.html,butter für die form,butter
1,104,/rezept/279085/Gyrosauflauf-mit-Gemuese.html,erbsen / möhren 420 gabtropfgewicht,pea
2,238,/rezept/372486/Zitronen-Schokoladenkuchen.html,zironensaft das entsprichtca. 1 - 2 zitronen,lemon
3,238,/rezept/372486/Zitronen-Schokoladenkuchen.html,zitronensaft das entsprichtca. 4 - 5 zitronen,lemon
4,282,/rezept/330603/Spiegelei-auf-Kartoffelbett-mit...,zwiebel frisch fein gehackt,onion


In [12]:
one_ingrs=list(set(one_ingr['end_ingr'].values.tolist()))
len(one_ingrs)

377

In [13]:
two_ingr=pd.read_csv('~/work/food_flavor/kochbar_ingredient_preprocessing/datasets/long_ingr_data/two_ingr_data/two_ingr_done.csv',index_col=0)
two_ingr.shape

(1975, 6)

In [14]:
two_ingrs=list(set(two_ingr['lemma_ingr'].values.tolist()))
len(two_ingrs)

470

In [15]:
two_ingrs

['mozzarella_cheese',
 'mett',
 'shrimp',
 'grape',
 'peperoncini',
 'blood_orange',
 'senf',
 'parmesan_cheese',
 'cone_waffle',
 'beetroot',
 'herb',
 'orange_liqueur',
 'kale',
 'tarragon',
 'cookies_chocolate',
 'blood_orange_yogurt',
 'vanilla_powder',
 'kassler',
 'baiser ',
 'milk_pudding',
 'currant_jelly',
 'salmon_skin',
 'knoblauch',
 'cream_cheese',
 'ginger',
 'blood_sausage',
 'Mettwurst',
 'caramel',
 'Nürnberger_Bratwurst',
 'almond_flour',
 'mustard_sausage',
 'Crema_di_balsamico',
 'kool',
 'mortar',
 'brunch',
 'milk_chocolate',
 'Tafelspitz',
 'mare',
 'Pesto_alla_Genovese',
 'chili_powder',
 'spice',
 'curry_goa',
 'vegetable_broth',
 'cumin',
 'maggi',
 'tomatoes_gusto',
 'white_chocolate',
 'wheat_bread',
 'pig_liver',
 'Noilly_Prat',
 'Blumenkohl_Sud',
 'spicy_cheese',
 'paprika',
 'jam',
 'coffin',
 'corn_starch',
 'chili_sauce',
 'italian_herb',
 'kidney_bean',
 'nutmeg',
 'reddish',
 'Biskin_Spezial',
 'duck',
 'Würzcreme',
 'ketchup',
 'cereal',
 'feta_patro

In [16]:
done_ingr=list(set(one_ingrs+two_ingrs))
len(done_ingr)

710

In [17]:
done_ingr

['mozzarella_cheese',
 'mett',
 'shrimp',
 'grape',
 'peperoncini',
 'blood_orange',
 'pudding_powder',
 'Sardellenpasta',
 'chilil',
 'senf',
 'parmesan_cheese',
 'cone_waffle',
 'beetroot',
 'herb',
 'orange_liqueur',
 'kale',
 'Palatschinken',
 'tarragon',
 'cookies_chocolate',
 'blood_orange_yogurt',
 'cress',
 'vanilla_powder',
 'cattle',
 'kassler',
 'baiser ',
 'milk_pudding',
 'Wurstreste',
 'currant_jelly',
 'Yufka',
 'salmon_skin',
 'knoblauch',
 'cream_cheese',
 'ginger',
 'blood_sausage',
 'tangerine',
 'Mettwurst',
 'caramel',
 'Schnitzel',
 'Nürnberger_Bratwurst',
 'Vanillepuddingpulver',
 'almond_flour',
 'chip',
 'basilikum',
 'mustard_sausage',
 'fig',
 'Crema_di_balsamico',
 'kool',
 'mortar',
 'brunch',
 'milk_chocolate',
 'Tafelspitz',
 'lentil',
 'mare',
 'Pesto_alla_Genovese',
 'chili_powder',
 'spice',
 'curry_goa',
 'vegetable_broth',
 'cumin',
 'pepperoni',
 'korma',
 'maggi',
 'tomatoes_gusto',
 'Eckerlkäs',
 'white_chocolate',
 'wheat_bread',
 'Pilsner_bier',

In [18]:
done_two_ingr=[]
for ingr in done_ingr:
    if '_' in ingr:
        done_two_ingr.append(ingr)

In [19]:
len(done_two_ingr)

260

In [20]:
done_two_ingr

['mozzarella_cheese',
 'blood_orange',
 'pudding_powder',
 'parmesan_cheese',
 'cone_waffle',
 'orange_liqueur',
 'cookies_chocolate',
 'blood_orange_yogurt',
 'vanilla_powder',
 'milk_pudding',
 'currant_jelly',
 'salmon_skin',
 'cream_cheese',
 'blood_sausage',
 'Nürnberger_Bratwurst',
 'almond_flour',
 'mustard_sausage',
 'Crema_di_balsamico',
 'milk_chocolate',
 'Pesto_alla_Genovese',
 'chili_powder',
 'curry_goa',
 'vegetable_broth',
 'tomatoes_gusto',
 'white_chocolate',
 'wheat_bread',
 'Pilsner_bier',
 'pig_liver',
 'Noilly_Prat',
 'Blumenkohl_Sud',
 'spicy_cheese',
 'corn_starch',
 'chili_sauce',
 'italian_herb',
 'kidney_bean',
 'chili_öl',
 'Biskin_Spezial',
 'feta_patros',
 'Camembert_cheese',
 'hot_chutney',
 'smoked_salmon',
 'red_bean',
 'mozzarella_cheesee',
 'hot_sauce',
 'brunch_herb',
 'pasta_penne',
 'soup_herb',
 'tomato_with_herbs',
 'red_wine',
 'coriander_salt',
 'Temp_Linsen',
 'french_bean',
 'crunchy_sausage',
 'potato_flour',
 'miracel_whip',
 'seasoned_spic

In [21]:
long_ingr.head()

,recipe_id,recipe_href,name,ingredient_name,en_ingr
0,86439,/rezept/254348/Gelbflossen-Thunfisch-Tranchen-...,gelb flossen thunfischtranchen a ca. 150 gramm,gelb flossen thunfischtranchen a,yellow_fins_tuna
1,45230,/rezept/242380/Zimt-Apfel-Kuchen.html,rosinen nach geschmackeingeweicht in rum,rosinen nach geschmackeingeweicht in rum,raisins_soaked_rum
2,57005,/rezept/281248/Chilicreme.html,frischkäse geht auch mitfeta käse,frischkäse geht auch mitfeta käse,cream_cheese_goes_well_feta_cheese
3,53172,/rezept/405833/Weihnachtliche-Muffins-a-la-Kra...,milch zum anrühren derbuttercreme,milch zum anrühren derbuttercreme,milk_mixing_buttercream
4,85805,/rezept/448409/Nachtisch-Cranberry-Dessert-a-l...,weisse schokolade mit nuss,weisse schokolade mit nuss,white_chocolate_nuts
